This contains the following datasets:
1. Google
2. Unacast (no longer updated)
3. Descartes Lab (no longer updated)

This returns separate files for each source of mobility data

# Latest update on 3/6/2022

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Importing the FIPS file

In [2]:
# importing fips data. and rearranging the columns
fips = pd.read_excel("~/Documents/ra/HPC/HPC_datahub/Raw data/fips.xlsx")
fips = fips.iloc[:, [1, 3, 2, 4, 5]]

# Convert the 'integers' to integers in fips file.
for i in range(fips.shape[0]):
    if fips.iloc[i, 4] != 'unknown':
        fips.iloc[i, 4] = int(fips.iloc[i, 4])
        fips.iloc[i, 2] = int(fips.iloc[i, 2])

fips = fips.drop(fips[fips.fips == 0].index)
fips = fips.drop(fips[fips.fips == "unknown"].index)
fips.iloc[3142]
fips = fips.reset_index(drop=True)
fips = fips[0:3142]
fips.loc[fips["stfips"] == 11]

,stfips,stname,ctyfips,ctyname,fips
319,11,District of Columbia,1,District of Columbia,11001


# Google File

In [5]:
# Importing the google file from the server
google = pd.read_csv("https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=a88b56a24e1a1e25", low_memory = False)

In [6]:
google.columns

Index(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline'],
      dtype='object')

In [7]:
# selecting only the USA
google = google.loc[google["country_region_code"] == "US"]
google = google.reset_index(drop=True)

# deleting irrelevant columns
google = google.iloc[:, 2:]

# Renaming the columns
cols = google.columns
google.rename(columns={cols[0]:"stname", cols[1]:"ctyname", cols[3]:"retail", cols[4]:"grocery", cols[5]:"parks", cols[6]: "transit", cols[7]: "workplace", cols[8]:"residential"}, inplace=True)
google

,stname,ctyname,metro_area,retail,grocery,parks,transit,workplace,residential,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0
2,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-17,6.0,0.0,28.0,-9.0,-24.0,5.0
3,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-18,0.0,-1.0,6.0,1.0,0.0,1.0
4,NaN,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2020-02-19,2.0,0.0,8.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925553,Wyoming,Weston County,NaN,NaN,56045.0,ChIJd4Rqhed3YocR7ubT5-HgoJg,2022-02-25,NaN,NaN,NaN,NaN,-23.0,NaN
1925554,Wyoming,Weston County,NaN,NaN,56045.0,ChIJd4Rqhed3YocR7ubT5-HgoJg,2022-02-28,NaN,NaN,NaN,NaN,-27.0,NaN
1925555,Wyoming,Weston County,NaN,NaN,56045.0,ChIJd4Rqhed3YocR7ubT5-HgoJg,2022-03-01,NaN,NaN,NaN,NaN,-22.0,NaN
1925556,Wyoming,Weston County,NaN,NaN,56045.0,ChIJd4Rqhed3YocR7ubT5-HgoJg,2022-03-02,NaN,NaN,NaN,NaN,-29.0,NaN


In [8]:
# Reshaping the file.
pivot = pd.pivot_table(google, index=["stname", "ctyname"], columns=["transit"], values=google.columns[3:], aggfunc='first')

In [9]:
pivot

grocery                                   \
transit                   2020-02-15 2020-02-16 2020-02-17 2020-02-18   
stname  ctyname                                                         
Alabama Autauga County        1001.0     1001.0     1001.0     1001.0   
        Baldwin County        1003.0     1003.0     1003.0     1003.0   
        Barbour County        1005.0     1005.0     1005.0     1005.0   
        Bibb County           1007.0     1007.0     1007.0     1007.0   
        Blount County         1009.0     1009.0     1009.0     1009.0   
...                              ...        ...        ...        ...   
Wyoming Sweetwater County    56037.0    56037.0    56037.0    56037.0   
        Teton County         56039.0    56039.0    56039.0    56039.0   
        Uinta County         56041.0    56041.0    56041.0    56041.0   
        Washakie County      56043.0        NaN    56043.0    56043.0   
        Weston County            NaN    56045.0    56045.0    56045.0   

                                                                       \
transit                   2020-02-19 2020-02-20 2020-02-21 2020-02-22   
stname  ctyname                                                         
Alabama Autauga County        1001.0     1001.0     1001.0     1001.0   
        Baldwin County        1003.0     1003.0     1003.0     1003.0   
        Barbour County        1005.0     1005.0     1005.0     1005.0   
        Bibb County           1007.0     1007.0     1007.0     1007.0   
        Blount County         1009.0     1009.0     1009.0     1009.0   
...                              ...        ...        ...        ...   
Wyoming Sweetwater County    56037.0    56037.0    56037.0    56037.0   
        Teton County         56039.0    56039.0    56039.0    56039.0   
        Uinta County         56041.0    56041.0    56041.0    56041.0   
        Washakie County      56043.0    56043.0    56043.0    56043.0   
        Weston County        56045.0    56045.0    56045.0        NaN   

                                                 ...  \
transit                   2020-02-23 2020-02-24  ...   
stname  ctyname                                  ...   
Alabama Autauga County        1001.0     1001.0  ...   
        Baldwin County        1003.0     1003.0  ...   
        Barbour County        1005.0     1005.0  ...   
        Bibb County           1007.0     1007.0  ...   
        Blount County         1009.0     1009.0  ...   
...                              ...        ...  ...   
Wyoming Sweetwater County    56037.0    56037.0  ...   
        Teton County         56039.0    56039.0  ...   
        Uinta County         56041.0    56041.0  ...   
        Washakie County          NaN    56043.0  ...   
        Weston County            NaN    56045.0  ...   

                          workplaces_percent_change_from_baseline             \
transit                                                2022-02-22 2022-02-23   
stname  ctyname                                                                
Alabama Autauga County                                      -15.0       -9.0   
        Baldwin County                                       -5.0       -1.0   
        Barbour County                                        2.0        5.0   
        Bibb County                                           2.0       -1.0   
        Blount County                                       -16.0      -13.0   
...                                                           ...        ...   
Wyoming Sweetwater County                                   -16.0      -14.0   
        Teton County                                        -14.0      -18.0   
        Uinta County                                        -16.0      -17.0   
        Washakie County                                      -3.0       -6.0   
        Weston County                                       -24.0      -28.0   

                                                                       \
tran

In [10]:
# Changing the column names as per the rule: variable_yyyymmdd
pivot.columns = [f'{i}_{datetime.strptime(j, "%Y-%m-%d").strftime("%Y%m%d")}' for i, j in pivot.columns]
pivot.columns = pivot.columns.map(''.join)
                 
# Reseting the index to get rid of the multi-level indexing
pivot = pivot.reset_index()
#pivot

In [11]:
for i in range(pivot.shape[0]):
    if fips.loc[(fips["stname"] == pivot.iloc[i, 0]) & (fips["ctyname"] == pivot.iloc[i, 1])].empty == True:
        for county in fips.loc[fips["stname"] == pivot.iloc[i, 0], "ctyname"]:
            
            
            if pivot.iloc[i, 0] == "District of Columbia":
                pivot.iloc[i, 1] = "District of Columbia"
                
            
            city_county = ["Baltimore", "St. Louis", "Fairfax", "Richmond", "Roanoke"]
            
            if pivot.iloc[i, 1] in city_county:
                pivot.iloc[i, 1] = pivot.iloc[i, 1] + " city"
            
            if pivot.iloc[i, 1] in county:
                pivot.iloc[i, 1] = county
                break
            if pivot.iloc[i, 1] == "La Salle Parish":
                pivot.iloc[i, 1] = "La Salle County"
            if pivot.iloc[i, 1] == "DoÃ±a Ana County":
                pivot.iloc[i, 1] = "Doña Ana County"

In [12]:
# Generating a google only final data frame
google_only = pd.merge(fips, pivot, on=["stname", "ctyname"], how="left")
google_only

,stfips,stname,ctyfips,ctyname,fips,grocery_20200215,grocery_20200216,grocery_20200217,grocery_20200218,grocery_20200219,...,workplaces_percent_change_from_baseline_20220222,workplaces_percent_change_from_baseline_20220223,workplaces_percent_change_from_baseline_20220224,workplaces_percent_change_from_baseline_20220225,workplaces_percent_change_from_baseline_20220226,workplaces_percent_change_from_baseline_20220227,workplaces_percent_change_from_baseline_20220228,workplaces_percent_change_from_baseline_20220301,workplaces_percent_change_from_baseline_20220302,workplaces_percent_change_from_baseline_20220303
0,1,Alabama,1,Autauga County,1001,1001.0,1001.0,1001.0,1001.0,1001.0,...,-15.0,-9.0,-8.0,-10.0,8.0,12.0,-8.0,-11.0,-9.0,-8.0
1,1,Alabama,3,Baldwin County,1003,1003.0,1003.0,1003.0,1003.0,1003.0,...,-5.0,-1.0,-1.0,-3.0,4.0,-1.0,-18.0,-26.0,-10.0,-8.0
2,1,Alabama,5,Barbour County,1005,1005.0,1005.0,1005.0,1005.0,1005.0,...,2.0,5.0,6.0,11.0,-5.0,-14.0,7.0,4.0,9.0,6.0
3,1,Alabama,7,Bibb County,1007,1007.0,1007.0,1007.0,1007.0,1007.0,...,2.0,-1.0,4.0,7.0,7.0,-24.0,4.0,7.0,6.0,7.0
4,1,Alabama,9,Blount County,1009,1009.0,1009.0,1009.0,1009.0,1009.0,...,-16.0,-13.0,-14.0,-12.0,-8.0,-12.0,-11.0,-17.0,-14.0,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56,Wyoming,37,Sweetwater County,56037,56037.0,56037.0,56037.0,56037.0,56037.0,...,-16.0,-14.0,-13.0,-13.0,-13.0,-21.0,-14.0,-14.0,-10.0,-12.0
3139,56,Wyoming,39,Teton County,56039,56039.0,56039.0,56039.0,56039.0,56039.0,...,-14.0,-18.0,-15.0,-6.0,-12.0,-17.0,-16.0,-16.0,-18.0,-17.0
3140,56,Wyoming,41,Uinta County,56041,56041.0,56041.0,56041.0,56041.0,56041.0,...,-16.0,-17.0,-14.0,-13.0,-7.0,-14.0,-15.0,-16.0,-19.0,-16.0
3141,56,Wyoming,43,Washakie County,56043,56043.0,NaN,56043.0,56043.0,56043.0,...,-3.0,-6.0,-3.0,-2.0,NaN,NaN,-5.0,-7.0,-11.0,-10.0


In [13]:
merged_set = set(google_only.iloc[:, 4].tolist())
pivot_set = set(pivot.iloc[:, 1].tolist())
google_only

,stfips,stname,ctyfips,ctyname,fips,grocery_20200215,grocery_20200216,grocery_20200217,grocery_20200218,grocery_20200219,...,workplaces_percent_change_from_baseline_20220222,workplaces_percent_change_from_baseline_20220223,workplaces_percent_change_from_baseline_20220224,workplaces_percent_change_from_baseline_20220225,workplaces_percent_change_from_baseline_20220226,workplaces_percent_change_from_baseline_20220227,workplaces_percent_change_from_baseline_20220228,workplaces_percent_change_from_baseline_20220301,workplaces_percent_change_from_baseline_20220302,workplaces_percent_change_from_baseline_20220303
0,1,Alabama,1,Autauga County,1001,1001.0,1001.0,1001.0,1001.0,1001.0,...,-15.0,-9.0,-8.0,-10.0,8.0,12.0,-8.0,-11.0,-9.0,-8.0
1,1,Alabama,3,Baldwin County,1003,1003.0,1003.0,1003.0,1003.0,1003.0,...,-5.0,-1.0,-1.0,-3.0,4.0,-1.0,-18.0,-26.0,-10.0,-8.0
2,1,Alabama,5,Barbour County,1005,1005.0,1005.0,1005.0,1005.0,1005.0,...,2.0,5.0,6.0,11.0,-5.0,-14.0,7.0,4.0,9.0,6.0
3,1,Alabama,7,Bibb County,1007,1007.0,1007.0,1007.0,1007.0,1007.0,...,2.0,-1.0,4.0,7.0,7.0,-24.0,4.0,7.0,6.0,7.0
4,1,Alabama,9,Blount County,1009,1009.0,1009.0,1009.0,1009.0,1009.0,...,-16.0,-13.0,-14.0,-12.0,-8.0,-12.0,-11.0,-17.0,-14.0,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56,Wyoming,37,Sweetwater County,56037,56037.0,56037.0,56037.0,56037.0,56037.0,...,-16.0,-14.0,-13.0,-13.0,-13.0,-21.0,-14.0,-14.0,-10.0,-12.0
3139,56,Wyoming,39,Teton County,56039,56039.0,56039.0,56039.0,56039.0,56039.0,...,-14.0,-18.0,-15.0,-6.0,-12.0,-17.0,-16.0,-16.0,-18.0,-17.0
3140,56,Wyoming,41,Uinta County,56041,56041.0,56041.0,56041.0,56041.0,56041.0,...,-16.0,-17.0,-14.0,-13.0,-7.0,-14.0,-15.0,-16.0,-19.0,-16.0
3141,56,Wyoming,43,Washakie County,56043,56043.0,NaN,56043.0,56043.0,56043.0,...,-3.0,-6.0,-3.0,-2.0,NaN,NaN,-5.0,-7.0,-11.0,-10.0


In [14]:
merged = (google_only.iloc[:, 3].tolist())
original = (pivot.iloc[:, 1].tolist())
mismatch = list(set(original) - set(merged))
mismatch


[]

In [16]:
# output
google_only.to_csv('../Pandemic/mobility_google.csv', index = False)

# Archived code for no longer updating sources: Unacast and Descartes
No need to run

## Unacast data

In [13]:
# Importing the unacast data. This has to be downloaded as a gzip file from the unacast google drive. 
# Hence cannot be taken directly from the internet link.
df = pd.read_csv("~/Documents/ra/HPC/HPC_datahub/Raw data/sds-v3-full-county.csv")

# renaming the "county_fips" to "fips"
df.rename(columns={"county_fips":"fips"}, inplace=True)

# rearranging the columns for pivoting.
col = [7, 3, 5, 6] + [i for i in range(8, 21)]
df = df.iloc[:, col]
#df

In [14]:
# pivoting
pivot = pd.pivot_table(df, index=["fips"], columns=["date"], values=df.columns[2:], aggfunc='first', dropna=False)

# changing the variable names to variable_date format
pivot.columns = [f'{i}_{datetime.strptime(j, "%Y-%m-%d").strftime("%Y%m%d")}' for i, j in pivot.columns]
pivot.columns = pivot.columns.map(''.join)
                 
# reseting the index to get rid of the multi level indexing
pivot = pivot.reset_index()

#pivot

In [15]:
# Making the unacast only dataset. Standalone and complete
unacast_only = pd.merge(fips, pivot, on=["fips"], how="left")

# Deleting the "county_population_date" and replace it with "county_population_2018"
todel = []
count = 0
for col in unacast_only.columns:
    if col.split("_")[0] == "county" and col.split("_")[1] == "population":
        count = count + 1
        if count == 1:
            unacast_only.rename(columns={col:"county_population_2018"}, inplace=True)
        else:
            del unacast_only[col]
#print(count)
unacast_only


,stfips,stname,ctyfips,ctyname,fips,county_population_2018,covid_20200224,covid_20200225,covid_20200226,covid_20200227,...,weekday_20200514,weekday_20200515,weekday_20200516,weekday_20200517,weekday_20200518,weekday_20200519,weekday_20200520,weekday_20200521,weekday_20200522,weekday_20200523
0,1,Alabama,1,Autauga County,1001,55601.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
1,1,Alabama,3,Baldwin County,1003,218022.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
2,1,Alabama,5,Barbour County,1005,24881.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
3,1,Alabama,7,Bibb County,1007,22400.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
4,1,Alabama,9,Blount County,1009,57840.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56,Wyoming,37,Sweetwater County,56037,43051.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
3138,56,Wyoming,39,Teton County,56039,23081.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
3139,56,Wyoming,41,Uinta County,56041,20299.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
3140,56,Wyoming,43,Washakie County,56043,7885.0,PRE,PRE,PRE,PRE,...,5.0,6.0,7.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0


In [ ]:
# output
unacast_only.to_csv('../Pandemic/mobility_unacast.csv')

## Descartes Lab data

In [16]:
# Importing the descartes lab data
sample = pd.read_csv("https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-mobility-daterow.csv")

# Removing unnecessary columns
col = [0, 3, 4, 5, 6, 7, 8]
sample = sample.iloc[:, col]

In [17]:
# Pivot
pivot = pd.pivot_table(sample, index=["admin1", "admin2"], columns=["date"], values=sample.columns[4:], aggfunc='first', dropna=False)

# Changing the name of the variables as per variable_date
pivot.columns = [f'{i}_{datetime.strptime(j, "%Y-%m-%d").strftime("%Y%m%d")}' for i, j in pivot.columns]
pivot.columns = pivot.columns.map(''.join)

# Reseting the index to remove the multi dimension indexing                 
pivot = pivot.reset_index()

# Renameing the columns stname and ctyname
pivot.rename(columns={"admin1":"stname", "admin2":"ctyname"}, inplace=True)

#pivot

In [18]:
pivot

,stname,ctyname,m50_20200301,m50_20200302,m50_20200303,m50_20200304,m50_20200305,m50_20200306,m50_20200307,m50_20200308,...,samples_20210410,samples_20210411,samples_20210413,samples_20210414,samples_20210415,samples_20210416,samples_20210417,samples_20210418,samples_20210419,samples_20210420
0,Alabama,Abbeville County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,Acadia Parish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,Accomack County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alabama,Ada County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,Adair County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81345,Wyoming,Young County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81346,Wyoming,Yuba County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81347,Wyoming,Yuma County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81348,Wyoming,Zapata County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Descartes data only. Standalone
descartes_only = pd.merge(fips, pivot, on=["stname", "ctyname"], how="left")
#descartes_only

In [20]:
# output
descartes_only.to_csv('../Pandemic/mobility_descartes.csv')